### TF Agents 사용

https://www.tensorflow.org/agents/tutorials/2_environments_tutorial?hl=ko#introduction

In [ ]:
!pip install tf_agents
import tf_agents

from ale_py import ALEInterface

ale = ALEInterface()

In [20]:
import random
# gym 사용
import gym
import numpy as np

In [33]:
from tf_agents.environments import py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.environments import utils
from tf_agents.specs import array_spec
from tf_agents.environments import wrappers
from tf_agents.environments import suite_gym
from tf_agents.trajectories import time_step as ts

In [31]:
# 블랙잭
class CardGameEnv(py_environment.PyEnvironment):
  # 각 계수 초기화. 0에서 1까지 행동을 정함
  def __init__(self):
    self._action_spec = array_spec.BoundedArraySpec(
        shape=(), dtype=np.int32, minimum=0, maximum=1, name='action')
    self._observation_spec = array_spec.BoundedArraySpec(
        shape=(1,), dtype=np.int32, minimum=0, name='observation')
    self._state = 0
    self._episode_ended = False
  # 행동 값. 0 아니면 1
  def action_spec(self):
    return self._action_spec
  # 현재 라운드에서 카드 합
  def observation_spec(self):
    return self._observation_spec
  # 초기화
  def _reset(self):
    self._state = 0
    self._episode_ended = False
    return ts.restart(np.array([self._state], dtype=np.int32))
  # step = 라운드
  def _step(self, action):

    if self._episode_ended:
      # 액션이 끝나면 에피소드도 끝난 것. 현재 행동은 무시
      # 에피소드 초기화
      return self.reset()

    # 행동 : 0일 경우 새 카드 받기. 1일 경우 현재 라운드 종료
    if action == 1:
      self._episode_ended = True
    elif action == 0:
      new_card = np.random.randint(1, 11)
      self._state += new_card
    else:
      raise ValueError('`action` should be 0 or 1.')
    # 21에 가까울수록 높은 리워드
    if self._episode_ended or self._state >= 21:
      reward = self._state - 21 if self._state <= 21 else -21
      return ts.termination(np.array([self._state], dtype=np.int32), reward)
    else:
      return ts.transition(
          np.array([self._state], dtype=np.int32), reward=0.0, discount=1.0)

In [34]:
# 에피소드 5개
environment = CardGameEnv()
utils.validate_py_environment(environment, episodes=5)

In [35]:
# 새 카드 뽑기/ 라운드 종료
get_new_card_action = np.array(0, dtype=np.int32)
end_round_action = np.array(1, dtype = np.int32)

# 한번의 스텝
time_step = environment.reset()
print(time_step)

TimeStep(
{'discount': array(1., dtype=float32),
 'observation': array([0], dtype=int32),
 'reward': array(0., dtype=float32),
 'step_type': array(0, dtype=int32)})


In [ ]:
# 끝나면 리워드